In [1]:
# General libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Models libraries
from sklearn import svm
# Models tuning libraries
from sklearn.model_selection import GridSearchCV

## Preliminary work on the dataset

In [2]:
yahoo_df = pd.read_csv("../data/NSQ.csv")
yahoo_df = yahoo_df.set_index("Date")

X = yahoo_df[['ROC_30', '4 Day ROI', 'EMA_12', 'MACD_12_26_9', 'SRSI_30',
       'Williams_14', 'ATR_14', 'Previous_differenced']]
y = yahoo_df["Up down"]

# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)

# Normalize

In [3]:
def shift_norm(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [4]:
X_train_80 = shift_norm(X_train_80)
X_test = shift_norm(X_test)

In [5]:
X_train = shift_norm(X_train)
X_valid = shift_norm(X_valid)

# Support Vector Machine (SVC)

In [6]:
SVC = svm.SVC(random_state = 42, kernel='linear')

# Fit the model
SVC.fit(X_train, y_train)

# Predict
y_pred_train = SVC.predict(X_train)
y_pred_valid = SVC.predict(X_valid)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.53529 - Validation Accuracy: 0.55860


In [7]:
y_pred_test = SVC.predict(X_test)
# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
SVC_accuracy_test = test_acc

Test Accuracy: 0.55860


### Tuning parametres with GridSearchCV(VERY SLOW!!)

We make parameters tuning using the GridSearchCV instance. This one allows us to fit on a dataset all the possible combinations of parameter values. Moreover it evaluates and retaines the best combination of parameters.

# Applying Gridsearchcv for parameters tuning
# 'poly' --> molto lento
# Setting C parameters
parameters = [{'C': [0.1, 1.0, 10, 50, 100 ],
              'gamma': [1,0.1,0.001],
                'kernel': ['linear','rbf', 'sigmoid',"poly"]
              }] 
best_SVC = GridSearchCV(estimator = svm.SVC(),refit=True,verbose=2,
                             param_grid = parameters, scoring = 'accuracy')

# Fit the model
best_SVC.fit(X_train, y_train)

print(best_SVC.best_estimator_)

In [8]:
SVC = svm.SVC(C=10, gamma=1, kernel='linear',probability=True)

# Fit the model
SVC.fit(X_train_80, y_train_80)

# Predict
y_pred_train = SVC.predict(X_train)
y_pred_valid = SVC.predict(X_valid)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.53529 - Validation Accuracy: 0.55860


In [9]:
y_pred_test = SVC.predict(X_test)
y_prob = SVC.predict_proba(X_test)[:,1]

# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
SVC_accuracy_test = test_acc

Test Accuracy: 0.55860


# Predict on all data 

In [15]:
df_prediction = pd.read_csv("../data/model_accuracy_nsq.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])

df_prediction

,RF_pred,RF_accuracy,RF_prob,SVC_pred,SVC_accuracy,SVC_prob
0,1,0.523629,0.657543,1,0.558601,0.544082
1,0,0.523629,0.324182,1,0.558601,0.544076
2,0,0.523629,0.496864,1,0.558601,0.545460
3,1,0.523629,0.574333,1,0.558601,0.545009
4,0,0.523629,0.421569,1,0.558601,0.543872
...,...,...,...,...,...,...
1053,1,0.523629,0.679418,1,0.558601,0.539406
1054,1,0.523629,0.555801,1,0.558601,0.537904
1055,1,0.523629,0.573323,1,0.558601,0.540966
1056,1,0.523629,0.719503,1,0.558601,0.536726


In [11]:
df_prediction["SVC_pred"] = y_pred_test
df_prediction["SVC_accuracy"] = [SVC_accuracy_test for x in range(len(y_pred_test))]
df_prediction["SVC_prob"] = y_prob


In [12]:
df_prediction

,RF_pred,RF_accuracy,RF_prob,SVC_pred,SVC_accuracy,SVC_prob
0,1,0.523629,0.657543,1,0.558601,0.544082
1,0,0.523629,0.324182,1,0.558601,0.544076
2,0,0.523629,0.496864,1,0.558601,0.545460
3,1,0.523629,0.574333,1,0.558601,0.545009
4,0,0.523629,0.421569,1,0.558601,0.543872
...,...,...,...,...,...,...
1053,1,0.523629,0.679418,1,0.558601,0.539406
1054,1,0.523629,0.555801,1,0.558601,0.537904
1055,1,0.523629,0.573323,1,0.558601,0.540966
1056,1,0.523629,0.719503,1,0.558601,0.536726


In [13]:
df_prediction.to_csv("../data/model_accuracy_nsq.csv")

In [14]:
# Plot the confusion matrix
#conf_stat = confusion_matrix(y_test, y_pred_test)
#ut.plot_confusion_matrix(conf_stat, Linearmodel_nt.classes_)